In [ ]:
### Calibration
if mach == 0.6:
    dt_ref = 0.0001 * subsampling
    shed_period = 0.03

elif mach == 0.1:
    dt_ref = 0.001 * subsampling
    shed_period = 0.18

elif mach == 0.01:
    dt_ref = 0.02 * subsampling
    shed_period = 1.84

dim_factor = shed_period * 0.16 # strouhal
alpha = 0.2 # calibration parameter

# optimal parameters 
b1_opt = b1
b2_opt = b2
L1_opt = L1
L2_opt = L2
Q_opt = Q_

# computing delta denominator
delta_denominator = 0
delta_denominator += np.sum(np.square(b1))
delta_denominator += np.sum(np.square(b2))
delta_denominator += np.sum(np.square(L1))
delta_denominator += np.sum(np.square(L2))
delta_denominator += np.sum(np.square(Q_))

# computing epsilon denominator
epsilon_denominator = 0
err_zero = 0
t_cal = shed_period * 5
steps = int(t_cal / (dt * dim_factor))
for i in range(steps):
    sample = int((dt * dim_factor)/dt_ref) * i
    a = coeffs[:num,sample]
    a_dot = np.empty_like(a)
    for k in range(a_dot.shape[0]):
        a_dot[k] = (nu * b1[k] + b2[k] + np.inner((nu*L1[k,:]+L2[k,:]),a) + np.matmul(np.matmul(np.expand_dims(a,1).T,Q_[k]),np.expand_dims(a,1)))
    a_opt = (coeffs[:num,sample+1] - coeffs[:num,sample]) / dt_ref
    epsilon_denominator += np.sum(np.square(a_opt - a_dot))
    err_zero += np.sum(np.square(a_opt))

def epsilon():

    global Q_, L1, L2, b1, b2, nu, num
    global Q_opt, L1_opt, L2_opt, b1_opt, b2_opt
    
    # calibration period 
    t_cal = shed_period * 5
    steps = int(t_cal / (dt * dim_factor))
    
    numerator = 0
    
    for i in range(steps):
        sample = int((dt * dim_factor)/dt_ref) * i

        # optimal activation
        a = coeffs[:num,sample]
        # construct hypothesized activation flow
        a_dot_opt = np.empty_like(a)
        for k in range(a_dot.shape[0]):
            a_dot_opt[k] = (nu * b1_opt[k] + b2_opt[k] + np.inner((nu*L1_opt[k,:]+L2_opt[k,:]),a) + np.matmul(np.matmul(np.expand_dims(a,1).T,Q_opt[k]),np.expand_dims(a,1)))
        
        # optimal activation flow
        a_opt = (coeffs[:num,sample+1] - coeffs[:num,sample]) / dt_ref

        numerator += np.sum(np.square(a_opt - a_dot_opt))

    return numerator / epsilon_denominator

# Delta term
def delta():
    # normalized distance to hypothesized model
    global Q_, L1, L2, b1, b2, nu, num
    global Q_opt, L1_opt, L2_opt, b1_opt, b2_opt

    numerator = 0
    denom = 0

    # manual computation of components
    numerator += np.sum(np.square(b1_opt - b1))
    numerator += np.sum(np.square(b2_opt - b2))
    numerator += np.sum(np.square(L1_opt - L1))
    numerator += np.sum(np.square(L2_opt - L2))
    for i in range(num):
        numerator += np.sum(np.square(Q_opt[i] - Q_[i]))

    return numerator / delta_denominator

# gradient of f (opt_parameters)
def grad():
    global num

    derivative = np.zeros(num)
    for k in range(num):
        derivative[k] += 0

    return 0

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
ax.plot(coeffs[0,:],coeffs[1,:],linewidth=1, linestyle="dashed", label="Mode 0 over Mode 1")
ax.plot(coeffs[2,:],coeffs[3,:],linewidth=1, linestyle="dashed", label="Mode 2 over Mode 3")
#ax.plot(coeffs[4,:],coeffs[5,:],linewidth=1, linestyle="dashed", label="Mode 4 over Mode 5")
ax.set_xlabel("Mode x",size=15)
ax.set_ylabel("Mode y",size=15)
plt.legend()


fig, ax = plt.subplots(1,1,figsize=(8,8))
ax.plot(sol.y[0,-3000:len(t)],sol.y[1,-3000:len(t)],linewidth=1, linestyle="dashed", label="Mode 0 over Mode 1")
ax.plot(sol.y[2,-3000:len(t)],sol.y[3,-3000:len(t)],linewidth=1, linestyle="dashed", label="Mode 2 over Mode 3")
#ax.plot(sol.y[4,-1000:len(t)],sol.y[5,-1000:len(t)],linewidth=1, linestyle="dashed", label="Mode 4 over Mode 5")
ax.set_xlabel("Mode x",size=15)
ax.set_ylabel("Mode y",size=15)
plt.legend()

In [ ]:
# reconstruct flows
rec_len = 100
ssubsampling=20
rec = np.repeat(np.expand_dims(q_avg,1),rec_len,1)
for i in range(num):
    rec += np.outer(pod_modes[:,i],sol.y[i,-100*ssubsampling::ssubsampling])